# シンプルな画像分類器

このノートブックでは、事前学習済みのニューラルネットワークを使用して画像を分類する方法を紹介します。

**学べること:**
- 事前学習済みモデルの読み込みと使用方法
- 画像の前処理
- 画像に対する予測の実行
- 信頼度スコアの理解

**ユースケース:** 画像内のオブジェクトを識別する（例:「猫」、「犬」、「車」など）

---


## ステップ 1: 必要なライブラリをインポートする

必要なツールをインポートしましょう。まだすべてを理解していなくても心配しないでください！


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## ステップ 2: 事前学習済みモデルを読み込む

**MobileNetV2** を使用します。このニューラルネットワークは、すでに何百万もの画像で学習されています。

これは **転移学習** と呼ばれます。他の誰かが学習させたモデルを利用することです！


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## ステップ3: ヘルパー関数

モデル用の画像を読み込み、準備するための関数を作成しましょう。


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## ステップ 4: サンプル画像でテスト

インターネットから取得した画像を分類してみましょう！


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### 各画像を分類


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## ステップ5: 自分の画像を試してみよう！

以下のURLを、分類したい任意の画像URLに置き換えてください。


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 何が起こったのか？

1. **事前学習済みモデルを読み込んだ** - MobileNetV2は数百万枚の画像で学習されています
2. **画像を前処理した** - モデルに合わせてサイズ変更やフォーマットを行いました
3. **モデルが予測を行った** - 1000種類のオブジェクトクラスに対する確率を出力しました
4. **結果をデコードした** - 数値を人間が読めるラベルに変換しました

### 信頼スコアの理解

- **90-100%**: 非常に自信あり（ほぼ確実に正しい）
- **70-90%**: 自信あり（おそらく正しい）
- **50-70%**: 多少自信あり（正しい可能性あり）
- **50%未満**: あまり自信なし（不確実）

### なぜ予測が間違う可能性があるのか？

- **特殊な角度や照明** - モデルは一般的な写真で学習されています
- **複数のオブジェクト** - モデルは主なオブジェクト1つを想定しています
- **珍しいオブジェクト** - モデルは1000種類のカテゴリしか認識できません
- **低品質な画像** - ぼやけた画像や画質の悪い画像は認識が難しいです

---


## 🚀 次のステップ

1. **異なる画像を試してみる:**
   - [Unsplash](https://unsplash.com)で画像を探す
   - 右クリック → 「画像アドレスをコピー」でURLを取得

2. **実験してみる:**
   - 抽象的なアートではどうなる？
   - 異なる角度からの物体を認識できる？
   - 複数の物体をどう処理する？

3. **さらに学ぶ:**
   - [コンピュータビジョンのレッスン](../lessons/4-ComputerVision/README.md)を探求する
   - 自分で画像分類器をトレーニングする方法を学ぶ
   - CNN（畳み込みニューラルネットワーク）の仕組みを理解する

---

## 🎉 おめでとうございます！

最先端のニューラルネットワークを使って画像分類器を構築しました！

この技術は以下のような場面で活用されています:
- Googleフォト（写真の整理）
- 自動運転車（物体認識）
- 医療診断（X線画像の解析）
- 品質管理（欠陥の検出）

これからも探求し、学び続けてください！ 🚀



---

**免責事項**:  
この文書は、AI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当方は責任を負いません。
